In [54]:
import numpy as np
import pandas as pd
from pandas import DataFrame, pivot_table, read_csv

from rxtrade.data import add_amplitude_column, add_basic_columns
from rxtrade.utils import get_data_files

In [55]:
def add_stats_columns(ticker: str, file_path: str) -> DataFrame:
    df = read_csv(file_path)

    add_basic_columns(df)
    add_amplitude_column(ticker, df)

    df["Market Open"] = np.where(
        ((df["time"].dt.hour == 8) & (df["time"].dt.minute >= 30)) |
        ((df["time"].dt.hour == 9) & (df["time"].dt.minute < 15)),
        "O",
        "N"
    )

    df[f"Open ({ticker})"] = df["open"]
    df[f"Close ({ticker})"] = df["close"]
    df[f"High ({ticker})"] = df["high"]
    df[f"Low ({ticker})"] = df["low"]
    return df

In [56]:
def process_for_pivot(ticker: str, df: DataFrame) -> None:
    df[f"Amplitude ({ticker})"] = df[f"Amplitude ({ticker})"] / (df[f"Close ({ticker})"] / df[f"Close (NQ)"])

In [57]:
def make_pivot(tickers: list[str], df: DataFrame) -> DataFrame:
    return pivot_table(
        df,
        values=[f"High ({ticker})" for ticker in tickers] + [f"Low ({ticker})" for ticker in tickers],
        index=["Market Date"],
        aggfunc={
                    f"High ({ticker})": np.max for ticker in tickers
                } | {
                    f"Low ({ticker})": np.min for ticker in tickers
                },
        dropna=False,
    )

In [84]:
from IPython.core.display_functions import display

tickers = []
df = DataFrame(columns=["time"])

pd.options.display.max_rows = 999

for data_file in get_data_files("./archive/futures/**/*.csv"):
    if data_file.period != 1:
        continue

    tickers.append(data_file.ticker)
    ticker_df = add_stats_columns(data_file.ticker, data_file.file_path)
    ticker_df = ticker_df[ticker_df["Market Open"] == "O"]

    if len(df):
        df = df.merge(ticker_df, left_on="time", right_on="time")
    else:
        df = ticker_df

for ticker in tickers:
    process_for_pivot(ticker, df)

df_group = df.groupby("Market Date")

data = [
    [
        df_group[f"High ({ticker})"].idxmax(),
        df_group[f"Low ({ticker})"].idxmin(),
    ] for ticker in tickers
]
idx_df = DataFrame([item for sublist in data for item in sublist]).T
df_main = idx_df.merge(df_group.first(), on="Market Date", suffixes=(" (Index)", None))

THRESHOLD = 0.0015

MULT = {
    "YM": 0.5,
    "NQ": 2,
    "ES": 5,
}

for index, row in df_main.iterrows():
    print(row["Market Date_x"].date())

    for ticker in tickers:
        px_open = row[f"Open ({ticker})"]

        px_ticker_high_idx = row[f"High ({ticker}) (Index)"]
        px_ticker_low_idx = row[f"Low ({ticker}) (Index)"]

        px_market_open_high = df.iloc[px_ticker_high_idx][f"High ({ticker})"]
        px_market_open_low = df.iloc[px_ticker_low_idx][f"Low ({ticker})"]

        px_ticker_high_idx %= 45
        px_ticker_low_idx %= 45

        px_rate_open_high = abs(px_market_open_high / px_open - 1)
        px_rate_open_low = abs(px_market_open_low / px_open - 1)

        px_open_high = abs(px_market_open_high - px_open)
        px_open_low = abs(px_market_open_low - px_open)

        title = f"{ticker} " \
                f"(UP +{px_open_high:7.2f} ($ {px_open_high * MULT[ticker]:7.2f}) {px_rate_open_high:.2%} / " \
                f"DOWN -{px_open_low:7.2f} ($ {px_open_low * MULT[ticker]:7.2f}) {px_rate_open_low:.2%})"

        if px_rate_open_high > THRESHOLD and px_rate_open_low > THRESHOLD:
            if px_ticker_high_idx < px_ticker_low_idx:
                print(f"{title}: ^v (0 - {px_ticker_high_idx} - {px_ticker_low_idx})")
            else:
                print(f"{title}: v^ (0 - {px_ticker_low_idx} - {px_ticker_high_idx})")
        elif px_rate_open_high > THRESHOLD:
            print(f"{title}: / (0 - {px_ticker_high_idx})")
        elif px_rate_open_low > THRESHOLD:
            print(f"{title}: \ (0 - {px_ticker_low_idx})")
        else:
            print(f"{title}: - (< {THRESHOLD:.2%})")


    print()


2022-01-17
ES (UP +   5.25 ($   26.25) 0.11% / DOWN -   4.75 ($   23.75) 0.10%): - (< 0.15%)
NQ (UP +  14.00 ($   28.00) 0.09% / DOWN -  51.25 ($  102.50) 0.33%): \ (0 - 20)
YM (UP +  60.00 ($   30.00) 0.17% / DOWN -   1.00 ($    0.50) 0.00%): / (0 - 30)

2022-01-18
ES (UP +   2.50 ($   12.50) 0.05% / DOWN -  25.75 ($  128.75) 0.56%): \ (0 - 28)
NQ (UP +  87.00 ($  174.00) 0.57% / DOWN -  76.00 ($  152.00) 0.50%): v^ (0 - 6 - 41)
YM (UP +   7.00 ($    3.50) 0.02% / DOWN - 208.00 ($  104.00) 0.59%): \ (0 - 28)

2022-01-19
ES (UP +  15.00 ($   75.00) 0.33% / DOWN -   4.00 ($   20.00) 0.09%): / (0 - 9)
NQ (UP + 104.75 ($  209.50) 0.69% / DOWN -  16.00 ($   32.00) 0.10%): / (0 - 34)
YM (UP +  68.00 ($   34.00) 0.19% / DOWN -  88.00 ($   44.00) 0.25%): ^v (0 - 9 - 44)

2022-01-20
ES (UP +  33.00 ($  165.00) 0.73% / DOWN -   2.25 ($   11.25) 0.05%): / (0 - 29)
NQ (UP + 125.00 ($  250.00) 0.82% / DOWN -   8.25 ($   16.50) 0.05%): / (0 - 32)
YM (UP + 271.00 ($  135.50) 0.77% / DOWN -  15.00 ($